In [8]:
import numpy as np
import pandas as pd
import pymysql
import os, sys
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import warnings

warnings.filterwarnings('ignore')
basePath = "..\\dataset"

In [9]:
df = pd.read_csv(basePath+ "\\SalesTrend.csv")
df.head()
#len(df)

REPORT_DATE BANNER_NAME PRODUCT_BKEY      QLI  LAST_1_MONTH_AVG  \
0  2016-09-01   Carrefour   XCN0401200  2.12976           2.12976   
1  2016-09-01   Carrefour   XCN0405400  0.27540           0.27540   
2  2016-09-01   Carrefour   XCN0413900  4.19832           4.19832   
3  2016-09-01   Carrefour   XCN0417509  1.89414           1.89414   
4  2016-09-01   Carrefour   XCN0420000  3.58020           3.58020   

   LAST_1_MONTH_MIN  LAST_1_MONTH_MAX  LAST_1_MONTH_SUM  LAST_1_MONTH_MID  \
0           2.12976           2.12976           2.12976           2.12976   
1           0.27540           0.27540           0.27540           0.27540   
2           4.19832           4.19832           4.19832           4.19832   
3           1.89414           1.89414           1.89414           1.89414   
4           3.58020           3.58020           3.58020           3.58020   

   LAST_1_MONTH_MID25         ...           LAST_6_MONTH_MID  \
0             2.12976         ...                    2.12976   
1             0.27540         ...                    0.27540   
2             4.19832         ...                    4.19832   
3             1.89414         ...                    1.89414   
4             3.58020         ...                    3.58020   

   LAST_6_MONTH_MID25  LAST_6_MONTH_MID75  LAST_12_MONTH_AVG  \
0             2.12976             2.12976            2.12976   
1             0.27540             0.27540            0.27540   
2             4.19832             4.19832            4.19832   
3             1.89414             1.89414            1.89414   
4             3.58020             3.58020            3.58020   

   LAST_12_MONTH_MIN  LAST_12_MONTH_MAX  LAST_12_MONTH_SUM  LAST_12_MONTH_MID  \
0            2.12976            2.12976            2.12976            2.12976   
1            0.27540            0.27540            0.27540            0.27540   
2            4.19832            4.19832            4.19832            4.19832   
3            1.89414            1.89414            1.89414            1.89414   
4            3.58020            3.58020            3.58020            3.58020   

   LAST_12_MONTH_MID25  LAST_12_MONTH_MID75  
0              2.12976              2.12976  
1              0.27540              0.27540  
2              4.19832              4.19832  
3              1.89414              1.89414  
4              3.58020              3.58020  

[5 rows x 53 columns]

In [10]:
import datetime
def getDayNo(arr):
    count = len(arr)
    for i in range(count):
        dd = arr[i]
        dd = datetime.datetime.strptime(dd,"%Y-%m-%d")

        dayNo = dd.timetuple().tm_yday
        arr[i] = dayNo
        #print("date:", dd, "; dayNo: ", dayNo)

In [11]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
def plot_learning_curves(model, X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    train_errors, val_errors = [], []
    for m in range(1, len(X_train)):
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_val_predict = model.predict(X_val)
        train_errors.append(mean_squared_error(y_train_predict, y_train[:m]))
        val_errors.append(mean_squared_error(y_val_predict, y_val))
    plt.plot(np.sqrt(train_errors), "r-+", linewidth=2, label="train")
    plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="val")

#plot_learning_curves(rf, X_train, y_train)

In [12]:
def model_train(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    yHat = model.predict(X_test)
    #plt.figure(figsize=(15, 12))
    plt.plot(range(len(yHat)), yHat, "r-")
    plt.plot(range(len(y_test)), y_test, "b--")
    from sklearn.metrics import mean_squared_error
    mse = mean_squared_error(y_test, yHat)
    print("mse: ", mse, "rmse: ", np.sqrt(mse))
    #plot_learning_curves(rf, X_train, y_train)

In [13]:
# 获取banner聚合信息
filePath = os.path.join(basePath, "banner_group.csv")
df_banner = pd.read_csv(filePath)
#df_banner.head()

In [14]:
# 获取趋势聚合信息
filePath = os.path.join(basePath, "SalesTrend.csv")
df_trend = pd.read_csv(filePath)
df_trend_carre = df_trend[df_trend.BANNER_NAME == 'Carrefour']
df_trend_carre_agg = df_trend_carre.groupby('REPORT_DATE').mean().reset_index()
# df_trend_carre_agg.head(3)

In [15]:
print(len(df_banner), len(df_trend_carre_agg))
df_merge = df_banner.merge(df_trend_carre_agg, on=["REPORT_DATE"], how="inner")
df_merge.head(2)

654 654


REPORT_DATE BANNER_NAME  WEEK_OF_YEAR  MONTH_OF_YEAR  YEAR_OF_WEEK  \
0  2016-09-01   Carrefour            35              9          2016   
1  2016-09-02   Carrefour            35              9          2016   

   IS_VALENTINE  IS_TEACHER  IS_C_VALENTINE  IS_CHILDREN  IS_NEWYEAR  \
0           0.0         0.0             0.0          0.0         0.0   
1           0.0         0.0             0.0          0.0         0.0   

          ...           LAST_6_MONTH_MID  LAST_6_MONTH_MID25  \
0         ...                   0.632679            0.632679   
1         ...                   0.541714            0.541714   

   LAST_6_MONTH_MID75  LAST_12_MONTH_AVG  LAST_12_MONTH_MIN  \
0            0.632679           0.632679           0.632679   
1            0.541714           0.541714           0.331946   

   LAST_12_MONTH_MAX  LAST_12_MONTH_SUM  LAST_12_MONTH_MID  \
0           0.632679           0.632679           0.632679   
1           0.751482           1.090651           0.541714   

   LAST_12_MONTH_MID25  LAST_12_MONTH_MID75  
0             0.632679             0.632679  
1             0.541714             0.541714  

[2 rows x 66 columns]

In [16]:
corr_matrix = df_merge.corr()
for item in corr_matrix["QLI"].sort_values(ascending=False):
    print(item)
corr_matrix["QLI"].sort_values(ascending=False)

1.0
0.5038399615353507
0.3308834127720195
0.3093288410322333
0.3033234767086347
0.29117723997077166
0.24636221610922215
0.20982151664679854
0.2065610700041062
0.19418823979556468
0.1830778280729819
0.1725395374457244
0.15411779468019896
0.15075448890958168
0.1400722256519061
0.13621704474467491
0.12454574466693202
0.11968978930376074
0.11851995386365265
0.11099409569442127
0.10444842551785262
0.09823930502070745
0.0945579334936705
0.08827381082460665
0.08771429808503876
0.08509006763704893
0.07737713559687415
0.07669979520398956
0.07449952121420182
0.07304453804763134
0.0730113761242744
0.06385577886901
0.06326701558756719
0.0579254445884309
0.053863442412731534
0.052372530837975495
0.048381014826638424
0.03976202368680569
0.037600625547974
0.03622175201439872
0.031667942245609114
0.0304053617726414
0.0300637776225
0.02780992088755997
0.02703941640556998
0.026264847762450486
0.022402133635100397
0.01887185962735974
0.012104333347005142
0.008824498308687185
0.00704310999601308
0.0067449

QLI                    1.000000
IS_CHRISTMAS           0.503840
LAST_1_MONTH_AVG       0.330883
LAST_1_MONTH_MID75     0.309329
LAST_1_MONTH_MAX       0.303323
LAST_1_MONTH_SUM       0.291177
LAST_1_MONTH_MID       0.246362
LAST_2_MONTH_AVG       0.209822
LAST_1_MONTH_MID25     0.206561
LAST_2_MONTH_MAX       0.194188
LAST_2_MONTH_SUM       0.183078
LAST_2_MONTH_MID75     0.172540
WEEK_OF_YEAR           0.154118
MONTH_OF_YEAR          0.150754
LAST_2_MONTH_MID       0.140072
LAST_2_MONTH_MID25     0.136217
LAST_3_MONTH_AVG       0.124546
LAST_3_MONTH_MAX       0.119690
LAST_1_MONTH_MIN       0.118520
LAST_3_MONTH_MID25     0.110994
LAST_3_MONTH_SUM       0.104448
LAST_3_MONTH_MID75     0.098239
LAST_4_MONTH_AVG       0.094558
LAST_3_MONTH_MID       0.088274
LAST_4_MONTH_MID75     0.087714
IS_12                  0.085090
LAST_4_MONTH_MAX       0.077377
LAST_4_MONTH_MID       0.076700
LAST_4_MONTH_MID25     0.074500
LAST_2_MONTH_MIN       0.073045
                         ...   
LAST_5_M

In [17]:
labels = df_merge["QLI"].values
df_merge_clean = df_merge.copy()
getDayNo(df_merge_clean["REPORT_DATE"].values)
df_merge_clean = df_merge_clean.drop("BANNER_NAME", 1)
df_merge_clean = df_merge_clean.drop("QLI", 1)
df_merge_clean.head(1)

NameError: name 'df_merge' is not defined

In [18]:
# 尝试去掉相关度低的列，但是从rmse效果不是很明显
df_merge_clean = df_merge.filter(items=["WEEK_OF_YEAR","LAST_1_MONTH_AVG","LAST_1_MONTH_MID75","LAST_1_MONTH_MAX","LAST_1_MONTH_SUM",
                                  "LAST_1_MONTH_MID","LAST_2_MONTH_AVG","LAST_1_MONTH_MID25","LAST_2_MONTH_MAX",
                                  "LAST_2_MONTH_SUM","LAST_2_MONTH_MID75","MONTH_OF_YEAR","LAST_2_MONTH_MID",
                                  "LAST_2_MONTH_MID25","LAST_3_MONTH_AVG","LAST_3_MONTH_MAX","LAST_1_MONTH_MIN",
                                  "LAST_3_MONTH_MID25","LAST_3_MONTH_SUM"])

df_merge_clean.head()

NameError: name 'df_merge' is not defined

In [19]:
len(df_merge_clean.values)
X_train, X_test, y_train, y_test = train_test_split(df_merge_clean.values, labels, test_size=0.2, random_state=42)

NameError: name 'df_merge_clean' is not defined

In [20]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
model_train(lr, X_train, y_train, X_test, y_test)
# 正常            mse:  1.740521567096297 rmse:  1.3192882805119952
# 只是取高相关的列：mse:  1.6498981200989498 rmse:  1.28448360055664

NameError: name 'X_train' is not defined

In [ ]:
rf = RandomForestRegressor(random_state=42,min_samples_leaf=150)
model_train(rf, X_train, y_train, X_test, y_test)
# 正常            mse:  1.6457994000584257 rmse:  1.282887134575145
# 只是取高相关的列：mse:  1.80228974242949 rmse:  1.3424938519149687
# max_features=1 :mse:  1.3514654105490587 rmse:  1.1625254451189697(正常)
# max_features=1 :mse:  1.6524300339530966 rmse:  1.2854687992919536(去掉无关列)
# n_estimators=100:mse:  1.556241037092118 rmse:  1.2474939026272305
# min_samples_leaf=150: mse:  1.4474719092476627 rmse:  1.2031092673766846(如果加max_feature=1反而会略小)

In [ ]:
from sklearn.svm import SVR
svr =SVR()
model_train(svr, X_train, y_train, X_test, y_test)
# 正常            mse:  1.4925522793217523 rmse:  1.2217005686017144
# 只是取高相关的列：mse:  1.4464301776578847 rmse:  1.2026762563790327

In [ ]:
gbr = GradientBoostingRegressor()
model_train(gbr, X_train, y_train, X_test, y_test)
# 正常            mse:  1.7530168784915257 rmse:  1.324015437406802
# 只是取高相关的列：mse:  1.9172499682142525 rmse:  1.3846479582241302

In [21]:
abr = AdaBoostRegressor()
model_train(abr, X_train, y_train, X_test, y_test)
# 正常            mse:  1.7041799726923388 rmse:  1.3054424432706095
# 只是取高相关的列：mse:  1.9216638377206163 rmse:  1.3862409017629715

NameError: name 'X_train' is not defined